# **SDSS光谱筛选**

* 此输出为含有plate,mjd,fiberID与SDSS的subclass的csv表格
（SDSS的分类并不准确需要注意）
* 输入为SDSS存下的hdu文件，若需要输入fits文件需要先行打开为hdu文件

In [ ]:
import math
import os
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

import numpy as np
from astropy.io import fits

import scipy.interpolate as spi
from sklearn.preprocessing import MinMaxScaler
import cv2

from pathlib import Path
from tqdm import tqdm, trange

In [ ]:
DB = {
    4471 : 'He I'
};

DZ = {
    3933.7:'Ca II K',
    3968.5:'Ca II H',
};

In [ ]:
def maincheck(data, 
         check_lines_dict,
         delta = 3.5e-06,
         center_broading = 5,
         peripheral_broading = 15):
    check_lines = [list(check_lines_dict.keys()) ,list(check_lines_dict.values())]
    name = []
    data
    for i in trange(0,len(data)):
        p = Lines_check(data[i], 
                       check_lines, 
                       delta = delta, 
                       center_broading = center_broading, 
                       peripheral_broading = peripheral_broading)
        if p != -1:
            name.append(p)
    return name

In [ ]:
def Delta_cal(x,
              y,
              line,
              center_broading,
              peripheral_broading
             ):
    ave_c = sum(y[(line - center_broading < x)&(x < line + center_broading)]) / len(y[(line - center_broading < x)&(x < line + center_broading)])
            
    ave_p = ((sum(y[(line + center_broading < x)&(x < line + center_broading + peripheral_broading)]) 
             + sum(y[(line - center_broading - peripheral_broading < x)&(x < line - center_broading)])) 
             / (len(y[(line + center_broading < x)&(x < line + center_broading + peripheral_broading)]) 
                + len(y[(line - center_broading - peripheral_broading < x)&(x < line - center_broading)])))
    return (ave_p - ave_c) / (max(y) - min(y))

In [ ]:
def Lines_check(file,
                lines,
                delta,
                center_broading,
                peripheral_broading
               ):
    file_fits = file[0]
    x = 10**file_fits[1].data['loglam']
    y = file_fits[1].data['flux']
    z = file_fits[2].data['Z']
    p=file_fits[2].data['plate']
    m=file_fits[2].data['mjd']
    f=file_fits[2].data['fiberID']
    #ra=file_fits[2].data['ra']
    #dec=file_fits[2].data['dec']
    #sng = file_fits[0].header['SNRU']
    subclass = file_fits[2].data['subclass']
    file_fits.close()
    
    ipo = spi.splrep(x , y , k = 3)
    X = x
    Y = spi.splev(X, ipo)
    Y = cv2.GaussianBlur(src = Y, ksize = (29, 29), sigmaX = 5)
    
    #if sng > 10 and subclass == 'WD':
    line = []
    if z == -9999:
        z = 0
    for i in range(0,len(lines[0])):
        line.append(lines[0][i] / (z + 1))
    text = [int(p),int(m),int(f),subclass]
    flag = 0
    n = 0

    for i in range(0,len(line)):
        if min(x) <= line[i] <= max(x):
            n = n + 1
            delta0 = Delta_cal(x, y, line[i], center_broading, peripheral_broading)
            delta1 = Delta_cal(X, Y, line[i], center_broading, peripheral_broading)
            if delta0 >= delta and delta1 >= delta:
                flag = flag + 1
    if flag == n and n != 0:
        return text
    else:
        return -1

In [ ]:
name = maincheck(sp[2000:], DZ)

In [ ]:
import pandas as pd
 
column=['plate','mjd','fiberID','subclass'] # 列表对应每列的列名
 
test=pd.DataFrame(columns=column,data=name)
 
test.to_csv('C:/Users/Lenovo/Desktop/polluted white dwalf/sdss/last/test2.csv') # 如果生成excel，可以用to_excel